In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_colwidth = 200

In [3]:
from pyathena import connect
conn = connect(s3_staging_dir='s3://jm-dsprod-athena/data-scientist-workgroup', region_name='ap-south-1')

In [4]:
%load_ext athena

In [6]:
%%athena

with ob_base as (
    select *
    from
    (select prospective_user_id as user_id,
           min(date(user_saving_account_created_date)) as acc_created_date
    from transformations_db.jupiter_onboarding_master 
    where user_saving_account_created_date is not null 
    group by 1
    )

)

, fkyc_data as (
    select *, 
    case when time_taken < 30 then 'ETB'
    when time_taken >= 30 then 'VKYC' end as kyc_mode
    from 
    (
    select distinct prospective_user_id as user_id, 'FKYC' as flg,
    user_saving_account_created_date, full_kyc_completion_date,
    date_diff('minute', user_saving_account_created_date, full_kyc_completion_date) as time_taken
    from transformations_db.jupiter_onboarding_master
    where user_saving_account_type = 'jupiter.accounts.full'
    and user_saving_account_created_date is not null and full_kyc_completion_date is not null
    ) 
)

, balance as
(select *
from transformations_db.jupiter_account_balance
where date(update_date) = 
(select max(date(update_date)) from transformations_db.jupiter_account_balance)
-- and account_desc = 'Savings account for federal customer'
),

user_cohort
as
(
select *
from (
select user_id, update_date, account_id, cbs_txn_date, current_balance
, row_number() over(partition by user_id,account_id,update_date order by cbs_txn_date desc) as row_num
from balance 
) 
where  row_num = 1
order by update_date
)

, user_balances as
(
select *,
case when balance < 0 then 0 
else balance end as acc_balance
from
(
select user_id, update_date, sum(current_balance) as balance

from user_cohort
where row_num = 1
group by 1,2
)
)

, recon_users as
(select *
from transformations_db.jupiter_reconciliation_transaction_master as t1
where updated_at = (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
                   where t1.reconciliation_id = t2.reconciliation_id
and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
and (t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
        OR (account_id like '%XX%'and t2.reconciliation_status in ('RECONCILIATION_JUPITER_SUCCESS')))
))

, txn_data as (
    select customer_id as user_id, date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) as txn_date, 
    transaction_amount, transaction_channel, debit_card_txn_type, upi_mcc_code, reconciliation_id,
    transaction_type, bank_transfer_type, cbs_log_payer_particulars, payments_record_status,
    debit_card_msg_type
    from recon_users 
    group by 1,2,3,4,5,6,7,8,9,10,11,12
)

, txn_dataset as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') then reconciliation_id end) as txns_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') then reconciliation_id end) as txns_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') then reconciliation_id end) as txns_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') then reconciliation_id end) as txns_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') then reconciliation_id end) as txns_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') then reconciliation_id end) as txns_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') then reconciliation_id end) as txns_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') then reconciliation_id end) as txns_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') then reconciliation_id end) as txns_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') then reconciliation_id end) as txns_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') then reconciliation_id end) as txns_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') then reconciliation_id end) as txns_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') then reconciliation_id end) as txns_june22
    from txn_data
    group by 1
)

, off_app_deposit as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'DEPOSIT' and transaction_channel is null then reconciliation_id end) as off_app_deposit_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'DEPOSIT' and transaction_channel is null then reconciliation_id end) as off_app_deposit_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'DEPOSIT' and transaction_channel is null then reconciliation_id end) as off_app_deposit_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'DEPOSIT' and transaction_channel is null then reconciliation_id end) as off_app_deposit_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'DEPOSIT' and transaction_channel is null then reconciliation_id end) as off_app_deposit_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'DEPOSIT' and transaction_channel is null then reconciliation_id end) as off_app_deposit_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'DEPOSIT' and transaction_channel is null then reconciliation_id end) as off_app_deposit_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'DEPOSIT' and transaction_channel is null then reconciliation_id end) as off_app_deposit_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'DEPOSIT' and transaction_channel is null then reconciliation_id end) as off_app_deposit_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'DEPOSIT' and transaction_channel is null then reconciliation_id end) as off_app_deposit_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'DEPOSIT' and transaction_channel is null then reconciliation_id end) as off_app_deposit_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'DEPOSIT' and transaction_channel is null then reconciliation_id end) as off_app_deposit_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'DEPOSIT' and transaction_channel is null then reconciliation_id end) as off_app_deposit_june22
    from txn_data
    group by 1
)

, off_app_withdrawal as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_june22
    from txn_data
    group by 1
)


, add_money as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_june22
    from txn_data
    group by 1
)

, bt_deposit as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_june22
    from txn_data
    group by 1
)

, upi_deposit as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_june22
    from txn_data
    group by 1
)

, interest_credit as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_june22
    from txn_data
    group by 1
)

, atm_with as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_june22
    from txn_data
    group by 1
)

, pos_with as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_june22
    from txn_data
    group by 1
)

, ecom_with as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_june22
    from txn_data
    group by 1
)

, bt_with as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_june22
    from txn_data
    group by 1
)

, upi_with as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_june22
    from txn_data
    group by 1
)


select distinct a.user_id, b.flg, b.kyc_mode, c.acc_balance as acc_balance, d.*, e.*, f.*,g.*, h.*, i.*
from ob_base a
left join fkyc_data b on a.user_id = b.user_id
left join user_balances c on a.user_id = c.user_id
left join txn_dataset d on a.user_id = d.user_id
left join add_money e on a.user_id = e.user_id
left join atm_with f on a.user_id = f.user_id
left join interest_credit g on a.user_id = g.user_id
left join off_app_deposit h on a.user_id = h.user_id
left join off_app_withdrawal i on a.user_id = i.user_id
where a.acc_created_date between date('2021-06-01') and date('2021-06-30')

KeyboardInterrupt: 

In [ ]:
select distinct a.user_id, b.flg, c.balance as acc_balance, d.*
from ob_base a
left join fkyc_data b on a.user_id = b.user_id
left join user_balances c on a.user_id = c.user_id
left join txn_dataset d on a.user_id = d.user_id
where a.acc_created_date between date('2021-06-01') and date('2021-06-30')


In [23]:
%%athena

with recon_users as
(select *
from transformations_db.jupiter_reconciliation_transaction_master as t1
where updated_at = (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
                   where t1.reconciliation_id = t2.reconciliation_id
and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
and (t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
        OR (account_id like '%XX%'and t2.reconciliation_status in ('RECONCILIATION_JUPITER_SUCCESS')))
))

, txn_data as (
    select customer_id as user_id, date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) as txn_date, 
    transaction_amount, transaction_channel, debit_card_txn_type, upi_mcc_code, reconciliation_id,
    transaction_type
    from recon_users 
)

, add_money as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_june22
    from txn_data
    group by 1
)

select *
from txn_finer
order by 1
limit 5

,user_id,add_money_june,add_money_july,add_money_august,add_money_sept,add_money_oct,add_money_nov,add_money_dec,add_money_jan,add_money_feb,add_money_march,add_money_april,add_money_may,add_money_june22
0,00002718-fcb3-450f-aa13-fa3da58a4bac,0,0,0,0,0,0,0,0,0,0,0,0,0
1,00002db7-fbe1-470b-a5cb-7653e4d60022,0,0,0,0,0,0,0,0,0,0,0,4,0
2,0000594b-3285-4181-b68f-1f1b83d570d7,0,0,0,0,0,0,0,0,0,0,0,0,0
3,00005e26-597c-4240-b542-6bb87106a8dc,0,0,0,0,0,0,0,0,0,0,0,0,0
4,00006a50-cadb-4321-8ddc-a44124c2acae,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
, off_app_withdrawal as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_june22
    from txn_data
    group by 1
)


In [31]:
august = pd.read_sql('''


with ob_base as (
    select *
    from
    (select prospective_user_id as user_id,
           min(date(user_saving_account_created_date)) as acc_created_date
    from transformations_db.jupiter_onboarding_master 
    where user_saving_account_created_date is not null 
    group by 1
    )

)

, fkyc_data as (
    select distinct prospective_user_id as user_id, 'FKYC' as flg
    from transformations_db.jupiter_onboarding_master
    where user_saving_account_type = 'jupiter.accounts.full'
)

, balance as
(select *
from transformations_db.jupiter_account_balance
where date(update_date) = 
(select max(date(update_date)) from transformations_db.jupiter_account_balance)
-- and account_desc = 'Savings account for federal customer'
),

user_cohort
as
(
select *
from (
select user_id, update_date, account_id, cbs_txn_date, current_balance
, row_number() over(partition by user_id,account_id,update_date order by cbs_txn_date desc) as row_num
from balance 
) 
where  row_num = 1
order by update_date
)

, user_balances as
(
select user_id, update_date, sum(current_balance) as balance

from user_cohort
where row_num = 1
group by 1,2
)

, recon_users as
(select *
from transformations_db.jupiter_reconciliation_transaction_master as t1
where updated_at = (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
                   where t1.reconciliation_id = t2.reconciliation_id
and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
and (t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
        OR (account_id like '%XX%'and t2.reconciliation_status in ('RECONCILIATION_JUPITER_SUCCESS')))
))

, txn_data as (
    select customer_id as user_id, date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) as txn_date, 
    transaction_amount, transaction_channel, debit_card_txn_type, upi_mcc_code, reconciliation_id,
    transaction_type, bank_transfer_type
    from recon_users 
)

, txn_dataset as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') then reconciliation_id end) as txns_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') then reconciliation_id end) as txns_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') then reconciliation_id end) as txns_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') then reconciliation_id end) as txns_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') then reconciliation_id end) as txns_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') then reconciliation_id end) as txns_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') then reconciliation_id end) as txns_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') then reconciliation_id end) as txns_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') then reconciliation_id end) as txns_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') then reconciliation_id end) as txns_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') then reconciliation_id end) as txns_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') then reconciliation_id end) as txns_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') then reconciliation_id end) as txns_june22
    from txn_data
    group by 1
)

, add_money as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_june22
    from txn_data
    group by 1
)

, atm_with as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_june22
    from txn_data
    group by 1
)

, interest_credit as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_june22
    from txn_data
    group by 1
)


select distinct a.user_id, b.flg, c.balance as acc_balance, d.*, e.*, f.*,g.*
from ob_base a
left join fkyc_data b on a.user_id = b.user_id
left join user_balances c on a.user_id = c.user_id
left join txn_dataset d on a.user_id = d.user_id
left join add_money e on a.user_id = e.user_id
left join atm_with f on a.user_id = f.user_id
left join interest_credit g on a.user_id = g.user_id
where a.acc_created_date between date('2021-08-01') and date('2021-08-31')


''', conn)

In [32]:
len(august)

16168

In [33]:
august.head()

,user_id,flg,acc_balance,user_id,txns_june,txns_july,txns_august,txns_sept,txns_oct,txns_nov,txns_dec,txns_jan,txns_feb,txns_march,txns_april,txns_may,txns_june22,user_id,add_money_june,add_money_july,add_money_august,add_money_sept,add_money_oct,add_money_nov,add_money_dec,add_money_jan,add_money_feb,add_money_march,add_money_april,add_money_may,add_money_june22,user_id,atm_with_june,atm_with_july,atm_with_august,atm_with_sept,atm_with_oct,atm_with_nov,atm_with_dec,atm_with_jan,atm_with_feb,atm_with_march,atm_with_april,atm_with_may,atm_with_june22,user_id,interest_credit_june,interest_credit_july,interest_credit_august,interest_credit_sept,interest_credit_oct,interest_credit_nov,interest_credit_dec,interest_credit_jan,interest_credit_feb,interest_credit_march,interest_credit_april,interest_credit_may,interest_credit_june22
0,6e0b9e8e-9e02-4574-823b-ba87a49cd92d,None,0.00,6e0b9e8e-9e02-4574-823b-ba87a49cd92d,0.00,0.00,12.00,16.00,27.00,0.00,4.00,14.00,2.00,0.00,15.00,5.00,0.00,6e0b9e8e-9e02-4574-823b-ba87a49cd92d,0.00,0.00,0.00,0.00,2.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,6e0b9e8e-9e02-4574-823b-ba87a49cd92d,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6e0b9e8e-9e02-4574-823b-ba87a49cd92d,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0b288235-52f8-44fc-96c5-8c3daf323c1f,None,0.00,0b288235-52f8-44fc-96c5-8c3daf323c1f,0.00,0.00,6.00,0.00,13.00,5.00,2.00,0.00,2.00,4.00,0.00,0.00,0.00,0b288235-52f8-44fc-96c5-8c3daf323c1f,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0b288235-52f8-44fc-96c5-8c3daf323c1f,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0b288235-52f8-44fc-96c5-8c3daf323c1f,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,21cdf626-36e3-4c3d-9fd3-f0ea25fb783e,None,-149.00,21cdf626-36e3-4c3d-9fd3-f0ea25fb783e,0.00,0.00,6.00,21.00,33.00,60.00,47.00,29.00,47.00,39.00,34.00,80.00,20.00,21cdf626-36e3-4c3d-9fd3-f0ea25fb783e,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21cdf626-36e3-4c3d-9fd3-f0ea25fb783e,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21cdf626-36e3-4c3d-9fd3-f0ea25fb783e,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0d5c730e-b4db-41a7-83a4-61a22b4a1932,FKYC,0.00,0d5c730e-b4db-41a7-83a4-61a22b4a1932,0.00,0.00,3.00,10.00,60.00,28.00,81.00,33.00,29.00,10.00,18.00,34.00,1.00,0d5c730e-b4db-41a7-83a4-61a22b4a1932,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0d5c730e-b4db-41a7-83a4-61a22b4a1932,0.00,0.00,0.00,0.00,6.00,3.00,10.00,7.00,11.00,5.00,6.00,2.00,0.00,0d5c730e-b4db-41a7-83a4-61a22b4a1932,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
4,GwG-LVA-LLuH-dy6,FKYC,98.78,GwG-LVA-LLuH-dy6,0.00,0.00,12.00,7.00,7.00,9.00,49.00,53.00,39.00,71.00,74.00,68.00,19.00,GwG-LVA-LLuH-dy6,0.00,0.00,1.00,1.00,0.00,0.00,5.00,3.00,2.00,2.00,2.00,7.00,3.00,GwG-LVA-LLuH-dy6,0.00,0.00,0.00,0.00,1.00,0.00,3.00,1.00,0.00,1.00,4.00,7.00,2.00,GwG-LVA-LLuH-dy6,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00


In [44]:
%%athena

with etb as (
    select distinct userid as user_id, report_order_date
    from delta_lake_db.credit_bureau_summary
    where cast(bureau_score as int) between 1 and 6
    group by 1,2
)

, ob_base as (
    select *
    from
    (select prospective_user_id as user_id,
           user_saving_account_created_date as acc_created_date,
           full_kyc_completion_date as fkyc_date
    from transformations_db.jupiter_onboarding_master 
    where user_saving_account_created_date is not null and full_kyc_completion_date is not null
    group by 1,2,3
    )
)


(
select a.*, b.report_order_date, date_diff('second', acc_created_date, fkyc_date) as time_taken
from ob_base a
join etb b on a.user_id = b.user_id
where cast(b.report_order_date as date)  <= date(a.acc_created_date)
)

,user_id,acc_created_date,fkyc_date,report_order_date,time_taken
0,4cd70b26-e5a5-4a89-99ce-5558d5359b4c,2022-05-18 19:42:51,2022-05-18 19:42:56.783,2022-05-18,5
1,9b36c808-fc6a-457d-9089-1485e872ab11,2022-03-27 10:37:21,2022-04-08 10:57:50.060,2022-03-27,1038029
2,78fd0589-9115-4c3c-b2e4-ec492c37f5d5,2022-05-09 12:21:23,2022-05-11 04:46:51.386,2022-05-09,145528
3,b271d686-aadc-41e5-81b5-8688243db9fe,2022-05-11 22:02:05,2022-05-13 11:31:29.950,2022-05-11,134964
4,4e417199-1eb4-4166-8359-625bc3b26c18,2022-04-25 20:00:27,2022-04-28 14:27:24.662,2022-04-25,239217
5,7e682bce-aff3-4e7f-b878-ae8c46c0eca3,2022-05-18 08:19:07,2022-05-18 08:19:12.873,2022-05-18,5
6,3ed6b825-c5b3-4ce6-b474-cce475f9c307,2022-05-11 13:55:50,2022-05-11 13:55:55.906,2022-05-11,5
7,d7f9ac5c-c63b-4d25-b755-808e1fa23f29,2022-02-12 21:55:20,2022-05-30 22:06:40.136,2022-02-12,9245480
8,1ab45cb5-ee05-4ddd-b9bd-79b8b012e584,2022-04-28 01:52:10,2022-04-28 01:52:15.247,2022-04-28,5
9,67423db3-f798-45ed-9e5f-0d1859688ba1,2022-02-11 19:10:38,2022-02-17 19:46:55.241,2022-02-11,520577


In [48]:
1038029/(30*60*60)

9.61137962962963

In [45]:
data = pd.read_sql('''

with etb as (
    select distinct userid as user_id, report_order_date
    from delta_lake_db.credit_bureau_summary
    where cast(bureau_score as int) between 1 and 6
    group by 1,2
)

, ob_base as (
    select *
    from
    (select prospective_user_id as user_id,
           user_saving_account_created_date as acc_created_date,
           full_kyc_completion_date as fkyc_date
    from transformations_db.jupiter_onboarding_master 
    where user_saving_account_created_date is not null and full_kyc_completion_date is not null
    group by 1,2,3
    )
)

select a.*, b.report_order_date, date_diff('second', acc_created_date, fkyc_date) as time_taken
from ob_base a
join etb b on a.user_id = b.user_id
where cast(b.report_order_date as date)  <= date(a.acc_created_date)
''', conn)

In [46]:
len(data)

9919

__Check for Depsoits__

In [23]:
%%athena

with recon_users as
(select *
from transformations_db.jupiter_reconciliation_transaction_master as t1
where updated_at = (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
                   where t1.reconciliation_id = t2.reconciliation_id
and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
and (t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
        OR (account_id like '%XX%'and t2.reconciliation_status in ('RECONCILIATION_JUPITER_SUCCESS')))
))

, txn_data as (
    select customer_id as user_id, date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) as txn_date, 
    transaction_amount, transaction_channel, debit_card_txn_type, upi_mcc_code, reconciliation_id,
    transaction_type, bank_transfer_type, cbs_log_payer_particulars, debit_card_msg_type
    from recon_users 
)

select distinct txn_date,
count(distinct case when transaction_type = 'DEPOSIT' and (transaction_channel is null and lower(cbs_log_payer_particulars) not like '%refund%') then reconciliation_id end) as off_app_dep,
count(distinct case when transaction_type = 'DEPOSIT' and 
transaction_channel is not null then reconciliation_id end) as in_app_dep
from txn_data
where txn_date >= date('2022-05-15')
group by 1
order by 1
limit 10

,txn_date,off_app_dep,in_app_dep
0,2022-05-15,132788,27856
1,2022-05-16,140772,29774
2,2022-05-17,142010,29438
3,2022-05-18,146067,30472
4,2022-05-19,141627,29865
5,2022-05-20,142194,29522
6,2022-05-21,142464,28927
7,2022-05-22,135558,27707
8,2022-05-23,149217,26371
9,2022-05-24,143816,29028


In [7]:
149396+25717

175113

In [ ]:
149431+26.

In [16]:
%%athena

with recon_users as
(select *
from transformations_db.jupiter_reconciliation_transaction_master as t1
where updated_at = (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
                   where t1.reconciliation_id = t2.reconciliation_id
and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
and (t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
        OR (account_id like '%XX%'and t2.reconciliation_status in ('RECONCILIATION_JUPITER_SUCCESS')))
))

, txn_data as (
    select customer_id as user_id, date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) as txn_date, 
    transaction_amount, transaction_channel, debit_card_txn_type, upi_mcc_code, reconciliation_id,
    transaction_type, bank_transfer_type, cbs_log_payer_particulars, payments_record_status
    from recon_users 
    group by 1,2,3,4,5,6,7,8,9,10,11
)

select distinct txn_date,
count(distinct reconciliation_id) as ct
from txn_data
where transaction_type = 'DEPOSIT' and transaction_channel in ('ADD_MONEY', 'BANK_TRANSFER', 'UPI', '')
group by 1
order by 1 desc
limit 10

,txn_date,ct
0,2022-06-16,32545
1,2022-06-15,32817
2,2022-06-14,32874
3,2022-06-13,19471
4,2022-06-12,10373
5,2022-06-11,12985
6,2022-06-10,14214
7,2022-06-09,16500
8,2022-06-08,31903
9,2022-06-07,35908


In [21]:
%%athena

with recon_users as
(select *
from transformations_db.jupiter_reconciliation_transaction_master as t1
where updated_at = (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
                   where t1.reconciliation_id = t2.reconciliation_id
and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
))

, txn_data as (
    select customer_id as user_id, date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) as txn_date, 
    transaction_amount, transaction_channel, debit_card_txn_type, upi_mcc_code, reconciliation_id,
    transaction_type, bank_transfer_type, cbs_log_payer_particulars, payments_record_status,
    debit_card_msg_type, reconciliation_status
    from recon_users 
    group by 1,2,3,4,5,6,7,8,9,10,11,12,13
)

select distinct reconciliation_status, cbs_log_payer_particulars
from txn_data
where transaction_type = 'DEPOSIT' and transaction_channel  = 'UPI'
and (lower(cbs_log_payer_particulars) like '%reversal%'
or lower(cbs_log_payer_particulars) like '%refund%')
group by 1,2
order by 1
limit 10

,reconciliation_status,cbs_log_payer_particulars
0,RECONCILIATION_SUCCESSFUL,UPI IN/208563503284/swiggy.refunds@axisbank//0000
1,RECONCILIATION_SUCCESSFUL,UPI IN/214058090177/amazon.refunds@axisbank//0000
2,RECONCILIATION_SUCCESSFUL,UPI IN/215618430364/amazon.refunds@axisbank//0000
3,RECONCILIATION_SUCCESSFUL,UPI IN/214068125200/amazon.refunds@axisbank//0000
4,RECONCILIATION_SUCCESSFUL,UPI IN/209466890427/amazon.refunds@axisbank//0000
5,RECONCILIATION_SUCCESSFUL,UPI IN/213546343935/amazon.refunds@axisbank//0000
6,RECONCILIATION_SUCCESSFUL,UPI IN/210280226232/amazon.refunds@axisbank//0000
7,RECONCILIATION_SUCCESSFUL,UPI IN/210028924622/amazon.refunds@axisbank//0000
8,RECONCILIATION_SUCCESSFUL,UPI IN/205571230202/amazon.refunds@axisbank//0000
9,RECONCILIATION_SUCCESSFUL,UPI IN/205972606122/amazon.refunds@axisbank//0000


In [23]:
32545+251

32796

In [12]:
data_june = pd.read_sql('''

with ob_base as (
    select *
    from
    (select prospective_user_id as user_id,
           min(date(user_saving_account_created_date)) as acc_created_date
    from transformations_db.jupiter_onboarding_master 
    where user_saving_account_created_date is not null 
    group by 1
    )

)

, fkyc_data as (
    select *, 
    case when time_taken < 30 then 'ETB'
    when time_taken >= 30 then 'VKYC' end as kyc_mode
    from 
    (
    select distinct prospective_user_id as user_id, 'FKYC' as flg,
    user_saving_account_created_date, full_kyc_completion_date,
    date_diff('minute', user_saving_account_created_date, full_kyc_completion_date) as time_taken
    from transformations_db.jupiter_onboarding_master
    where user_saving_account_type = 'jupiter.accounts.full'
    and user_saving_account_created_date is not null and full_kyc_completion_date is not null
    ) 
)

, balance as
(select *
from transformations_db.jupiter_account_balance
where date(update_date) = 
(select max(date(update_date)) from transformations_db.jupiter_account_balance)
-- and account_desc = 'Savings account for federal customer'
),

user_cohort
as
(
select *
from (
select user_id, update_date, account_id, cbs_txn_date, current_balance
, row_number() over(partition by user_id,account_id,update_date order by cbs_txn_date desc) as row_num
from balance 
) 
where  row_num = 1
order by update_date
)

, user_balances as
(
select *,
case when balance < 0 then 0 
else balance end as acc_balance
from
(
select user_id, update_date, sum(current_balance) as balance

from user_cohort
where row_num = 1
group by 1,2
)
)

, recon_users as
(select *
from transformations_db.jupiter_reconciliation_transaction_master as t1
where updated_at = (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
                   where t1.reconciliation_id = t2.reconciliation_id
and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
and (t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
        OR (account_id like '%XX%'and t2.reconciliation_status in ('RECONCILIATION_JUPITER_SUCCESS')))
))

, txn_data as (
    select customer_id as user_id, date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) as txn_date, 
    transaction_amount, transaction_channel, debit_card_txn_type, upi_mcc_code, reconciliation_id,
    transaction_type, bank_transfer_type, cbs_log_payer_particulars, payments_record_status,
    debit_card_msg_type
    from recon_users 
    group by 1,2,3,4,5,6,7,8,9,10,11,12
)

, txn_dataset as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') then reconciliation_id end) as txns_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') then reconciliation_id end) as txns_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') then reconciliation_id end) as txns_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') then reconciliation_id end) as txns_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') then reconciliation_id end) as txns_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') then reconciliation_id end) as txns_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') then reconciliation_id end) as txns_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') then reconciliation_id end) as txns_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') then reconciliation_id end) as txns_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') then reconciliation_id end) as txns_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') then reconciliation_id end) as txns_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') then reconciliation_id end) as txns_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') then reconciliation_id end) as txns_june22
    from txn_data
    group by 1
)



, off_app_withdrawal as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel is null then reconciliation_id end) as off_app_withdrawal_june22
    from txn_data
    group by 1
)

, off_app_deposit as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and (bank_transfer_type != 'SB_INTEREST' or bank_transfer_type is null or bank_transfer_type in ('BRANCH_CASH_DEPOSIT', 'IMPS')) then reconciliation_id end) as off_app_deposit_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and (bank_transfer_type != 'SB_INTEREST' or bank_transfer_type is null or bank_transfer_type in ('BRANCH_CASH_DEPOSIT', 'IMPS')) then reconciliation_id end) as off_app_deposit_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and (bank_transfer_type != 'SB_INTEREST' or bank_transfer_type is null or bank_transfer_type in ('BRANCH_CASH_DEPOSIT', 'IMPS')) then reconciliation_id end) as off_app_deposit_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and (bank_transfer_type != 'SB_INTEREST' or bank_transfer_type is null or bank_transfer_type in ('BRANCH_CASH_DEPOSIT', 'IMPS')) then reconciliation_id end) as off_app_deposit_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and (bank_transfer_type != 'SB_INTEREST' or bank_transfer_type is null or bank_transfer_type in ('BRANCH_CASH_DEPOSIT', 'IMPS')) then reconciliation_id end) as off_app_deposit_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and (bank_transfer_type != 'SB_INTEREST' or bank_transfer_type is null or bank_transfer_type in ('BRANCH_CASH_DEPOSIT', 'IMPS')) then reconciliation_id end) as off_app_deposit_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and (bank_transfer_type != 'SB_INTEREST' or bank_transfer_type is null or bank_transfer_type in ('BRANCH_CASH_DEPOSIT', 'IMPS')) then reconciliation_id end) as off_app_deposit_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and (bank_transfer_type != 'SB_INTEREST' or bank_transfer_type is null or bank_transfer_type in ('BRANCH_CASH_DEPOSIT', 'IMPS')) then reconciliation_id end) as off_app_deposit_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and (bank_transfer_type != 'SB_INTEREST' or bank_transfer_type is null or bank_transfer_type in ('BRANCH_CASH_DEPOSIT', 'IMPS')) then reconciliation_id end) as off_app_deposit_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and (bank_transfer_type != 'SB_INTEREST' or bank_transfer_type is null or bank_transfer_type in ('BRANCH_CASH_DEPOSIT', 'IMPS')) then reconciliation_id end) as off_app_deposit_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and (bank_transfer_type != 'SB_INTEREST' or bank_transfer_type is null or bank_transfer_type in ('BRANCH_CASH_DEPOSIT', 'IMPS')) then reconciliation_id end) as off_app_deposit_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and (bank_transfer_type != 'SB_INTEREST' or bank_transfer_type is null or bank_transfer_type in ('BRANCH_CASH_DEPOSIT', 'IMPS')) then reconciliation_id end) as off_app_deposit_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and (bank_transfer_type != 'SB_INTEREST' or bank_transfer_type is null or bank_transfer_type in ('BRANCH_CASH_DEPOSIT', 'IMPS')) then reconciliation_id end) as off_app_deposit_june22
    from txn_data
    group by 1
)

, add_money as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'ADD_MONEY' then reconciliation_id end) as add_money_june22
    from txn_data
    group by 1
)

, bt_deposit as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'BANK_TRANSFER' then reconciliation_id end) as BANK_TRANSFER_june22
    from txn_data
    group by 1
)

, upi_deposit as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'DEPOSIT' and transaction_channel = 'UPI' then reconciliation_id end) as UPI_june22
    from txn_data
    group by 1
)

, interest_credit as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'DEPOSIT' and transaction_channel is null and bank_transfer_type = 'SB_INTEREST' then reconciliation_id end) as interest_credit_june22
    from txn_data
    group by 1
)

, atm_with as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ATM_WITH' then reconciliation_id end) as atm_with_june22
    from txn_data
    group by 1
)

, pos_with as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type in ('POS', 'CONTACTLESS_POS') then reconciliation_id end) as pos_june22
    from txn_data
    group by 1
)

, ecom_with as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'DEBIT_CARD' and debit_card_txn_type = 'ECOM' then reconciliation_id end) as ecom_june22
    from txn_data
    group by 1
)

, bt_with as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel in ('BANK_TRANSFER', 'MANDATE') then reconciliation_id end) as bt_june22
    from txn_data
    group by 1
)

, upi_with as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_type = 'WITHDRAWAL' and transaction_channel = 'UPI' then reconciliation_id end) as upi_with_june22
    from txn_data
    group by 1
)


select distinct a.user_id, b.flg, b.kyc_mode, c.acc_balance as acc_balance, d.*, e.*, f.*,g.*, h.*, i.*
from ob_base a
left join fkyc_data b on a.user_id = b.user_id
left join user_balances c on a.user_id = c.user_id
left join txn_dataset d on a.user_id = d.user_id
left join add_money e on a.user_id = e.user_id
left join atm_with f on a.user_id = f.user_id
left join interest_credit g on a.user_id = g.user_id
left join off_app_deposit h on a.user_id = h.user_id
left join off_app_withdrawal i on a.user_id = i.user_id
left join upi_deposit j on a.user_id = j.user_id
left join bt_deposit k on a.user_id = k.user_id
left join bt_deposit k on a.user_id = k.user_id
left join pos_with l on a.user_id = l.user_id
left join ecom_with m on a.user_id = m.user_id
left join bt_with n on a.user_id = n.user_id
left join upi_with o on a.user_id = o.user_id
where a.acc_created_date between date('2021-06-01') and date('2021-06-30')

''',conn)

DatabaseError: Execution failed on sql: 

with ob_base as (
    select *
    from
    (select prospective_user_id as user_id,
           min(date(user_saving_account_created_date)) as acc_created_date
    from transformations_db.jupiter_onboarding_master 
    where user_saving_account_created_date is not null 
    group by 1
    )

)

, fkyc_data as (
    select *, 
    case when time_taken < 30 then 'ETB'
    when time_taken >= 30 then 'VKYC' end as kyc_mode
    from 
    (
    select distinct prospective_user_id as user_id, 'FKYC' as flg,
    user_saving_account_created_date, full_kyc_completion_date,
    date_diff('minute', user_saving_account_created_date, full_kyc_completion_date) as time_taken
    from transformations_db.jupiter_onboarding_master
    where user_saving_account_type = 'jupiter.accounts.full'
    and user_saving_account_created_date is not null and full_kyc_completion_date is not null
    ) 
)

, balance as
(select *
from transformations_db.jupiter_account_balance
w

In [8]:
len(data_june)

308

In [9]:
data_june

,user_id,flg,kyc_mode,acc_balance,user_id,txns_june,txns_july,txns_august,txns_sept,txns_oct,txns_nov,txns_dec,txns_jan,txns_feb,txns_march,txns_april,txns_may,txns_june22,user_id,add_money_june,add_money_july,add_money_august,add_money_sept,add_money_oct,add_money_nov,add_money_dec,add_money_jan,add_money_feb,add_money_march,add_money_april,add_money_may,add_money_june22,user_id,atm_with_june,atm_with_july,atm_with_august,atm_with_sept,atm_with_oct,atm_with_nov,atm_with_dec,atm_with_jan,atm_with_feb,atm_with_march,atm_with_april,atm_with_may,atm_with_june22,user_id,interest_credit_june,interest_credit_july,interest_credit_august,interest_credit_sept,interest_credit_oct,interest_credit_nov,interest_credit_dec,interest_credit_jan,interest_credit_feb,interest_credit_march,interest_credit_april,interest_credit_may,interest_credit_june22,user_id,off_app_deposit_june,off_app_deposit_july,off_app_deposit_august,off_app_deposit_sept,off_app_deposit_oct,off_app_deposit_nov,off_app_deposit_dec,off_app_deposit_jan,off_app_deposit_feb,off_app_deposit_march,off_app_deposit_april,off_app_deposit_may,off_app_deposit_june22,user_id,off_app_withdrawal_june,off_app_withdrawal_july,off_app_withdrawal_august,off_app_withdrawal_sept,off_app_withdrawal_oct,off_app_withdrawal_nov,off_app_withdrawal_dec,off_app_withdrawal_jan,off_app_withdrawal_feb,off_app_withdrawal_march,off_app_withdrawal_april,off_app_withdrawal_may,off_app_withdrawal_june22
0,UXo-GlA-TmTK-wGK,FKYC,VKYC,30.31,UXo-GlA-TmTK-wGK,2.00,0.00,1.00,1.00,0.00,2.00,8.00,5.00,2.00,6.00,0.00,0.00,0.00,UXo-GlA-TmTK-wGK,1.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,1.00,2.00,0.00,0.00,0.00,UXo-GlA-TmTK-wGK,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,UXo-GlA-TmTK-wGK,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,UXo-GlA-TmTK-wGK,1.00,0.00,0.00,1.00,0.00,0.00,2.00,1.00,0.00,1.00,0.00,0.00,0.00,UXo-GlA-TmTK-wGK,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,SGD-bMs-BPgR-tMi,None,None,1857.74,SGD-bMs-BPgR-tMi,4.00,1.00,8.00,2.00,1.00,0.00,9.00,10.00,17.00,22.00,21.00,4.00,0.00,SGD-bMs-BPgR-tMi,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,1.00,3.00,0.00,0.00,SGD-bMs-BPgR-tMi,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,SGD-bMs-BPgR-tMi,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,SGD-bMs-BPgR-tMi,1.00,0.00,0.00,1.00,0.00,0.00,6.00,4.00,0.00,1.00,1.00,2.00,0.00,SGD-bMs-BPgR-tMi,0.00,0.00,5.00,0.00,1.00,0.00,3.00,6.00,2.00,14.00,8.00,2.00,0.00
2,eIh-iOx-cHAc-GQ5,None,None,0.00,eIh-iOx-cHAc-GQ5,5.00,12.00,0.00,3.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,eIh-iOx-cHAc-GQ5,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,eIh-iOx-cHAc-GQ5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,eIh-iOx-cHAc-GQ5,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,eIh-iOx-cHAc-GQ5,1.00,5.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,eIh-iOx-cHAc-GQ5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Bjv-u0M-kyug-eEt,FKYC,VKYC,10552.74,Bjv-u0M-kyug-eEt,0.00,0.00,4.00,23.00,32.00,25.00,16.00,6.00,10.00,12.00,9.00,3.00,3.00,Bjv-u0M-kyug-eEt,0.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,Bjv-u0M-kyug-eEt,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Bjv-u0M-kyug-eEt,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,Bjv-u0M-kyug-eEt,0.00,0.00,0.00,2.00,1.00,2.00,6.00,3.00,0.00,2.00,0.00,1.00,1.00,Bjv-u0M-kyug-eEt,0.00,0.00,0.00,1.00,0.00,1.00,4.00,0.00,0.00,0.00,1.00,0.00,0.00
4,93c282af-183b-4751-a973-18e8659056a6,FKYC,VKYC,179.00,93c282af-183b-4751-a973-18e8659056a6,2.00,0.00,4.00,1.00,0.00,0.00,1.00,0.00,1.00,1.00,0.00,0.00,0.00,93c282af-183b-4751-a973-18e8659056a6,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,93c282af-183b-4751-a973-18e8659056a6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,93c282af-183b-4751-a973-18e8659056a6,0.00,0.00,0.00,1.00,0.00,0.00,

In [31]:
data_june = pd.read_sql('''

with ob_base as (
    select *
    from
    (select prospective_user_id as user_id,
           min(date(user_saving_account_created_date)) as acc_created_date
    from transformations_db.jupiter_onboarding_master 
    where user_saving_account_created_date is not null 
    group by 1
    )

)

, fkyc_data as (
    select *, 
    case when time_taken < 30 then 'ETB'
    when time_taken >= 30 then 'VKYC' end as kyc_mode
    from 
    (
    select distinct prospective_user_id as user_id, 'FKYC' as flg,
    user_saving_account_created_date, full_kyc_completion_date,
    date_diff('minute', user_saving_account_created_date, full_kyc_completion_date) as time_taken
    from transformations_db.jupiter_onboarding_master
    where user_saving_account_type = 'jupiter.accounts.full'
    and user_saving_account_created_date is not null and full_kyc_completion_date is not null
    ) 
)

, balance as
(select *
from transformations_db.jupiter_account_balance
where date(update_date) = 
(select date('2022-06-15') from transformations_db.jupiter_account_balance)
-- and account_desc = 'Savings account for federal customer'
),

user_cohort
as
(
select *
from (
select user_id, update_date, account_id, cbs_txn_date, current_balance
, row_number() over(partition by user_id,account_id,update_date order by cbs_txn_date desc) as row_num
from balance 
) 
where  row_num = 1
order by update_date
)

, user_balances as
(
select *,
case when balance < 0 then 0 
else balance end as acc_balance
from
(
select user_id, update_date, sum(current_balance) as balance

from user_cohort
where row_num = 1
group by 1,2
)
)

, recon_users as
(select *
from transformations_db.jupiter_reconciliation_transaction_master as t1
where updated_at = (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
                   where t1.reconciliation_id = t2.reconciliation_id
and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
and (t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
        OR (account_id like '%XX%'and t2.reconciliation_status in ('RECONCILIATION_JUPITER_SUCCESS')))
))

, txn_data as (
    select customer_id as user_id, date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) as txn_date, 
    transaction_amount, transaction_channel, debit_card_txn_type, upi_mcc_code, reconciliation_id,
    transaction_type, bank_transfer_type, cbs_log_payer_particulars, payments_record_status,
    debit_card_msg_type
    from recon_users 
    group by 1,2,3,4,5,6,7,8,9,10,11,12
)

, off_app as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_channel is null then reconciliation_id end) as off_app_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_channel is null then reconciliation_id end) as off_app_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_channel is null then reconciliation_id end) as off_app_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_channel is null then reconciliation_id end) as off_app_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_channel is null then reconciliation_id end) as off_app_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_channel is null then reconciliation_id end) as off_app_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_channel is null then reconciliation_id end) as off_app_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_channel is null then reconciliation_id end) as off_app_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_channel is null then reconciliation_id end) as off_app_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_channel is null then reconciliation_id end) as off_app_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_channel is null then reconciliation_id end) as off_app_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_channel is null then reconciliation_id end) as off_app_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_channel is null then reconciliation_id end) as off_app_june22
    from txn_data
    group by 1
)

, in_app as 
(
    select distinct user_id,
    count(distinct case when date_trunc('month', txn_date) = date('2021-06-01') and transaction_channel is not null then reconciliation_id end) as in_app_june,
    count(distinct case when date_trunc('month', txn_date) = date('2021-07-01') and transaction_channel is not null then reconciliation_id end) as in_app_july,
    count(distinct case when date_trunc('month', txn_date) = date('2021-08-01') and transaction_channel is not null then reconciliation_id end) as in_app_august,
    count(distinct case when date_trunc('month', txn_date) = date('2021-09-01') and transaction_channel is not null then reconciliation_id end) as in_app_sept,
    count(distinct case when date_trunc('month', txn_date) = date('2021-10-01') and transaction_channel is not null then reconciliation_id end) as in_app_oct,
    count(distinct case when date_trunc('month', txn_date) = date('2021-11-01') and transaction_channel is not null then reconciliation_id end) as in_app_nov,
    count(distinct case when date_trunc('month', txn_date) = date('2021-12-01') and transaction_channel is not null then reconciliation_id end) as in_app_dec,
    count(distinct case when date_trunc('month', txn_date) = date('2022-01-01') and transaction_channel is not null then reconciliation_id end) as in_app_jan,
    count(distinct case when date_trunc('month', txn_date) = date('2022-02-01') and transaction_channel is not null then reconciliation_id end) as in_app_feb,
    count(distinct case when date_trunc('month', txn_date) = date('2022-03-01') and transaction_channel is not null then reconciliation_id end) as in_app_march,
    count(distinct case when date_trunc('month', txn_date) = date('2022-04-01') and transaction_channel is not null then reconciliation_id end) as in_app_april,
    count(distinct case when date_trunc('month', txn_date) = date('2022-05-01') and transaction_channel is not null then reconciliation_id end) as in_app_may,
    count(distinct case when date_trunc('month', txn_date) = date('2022-06-01') and transaction_channel is not null then reconciliation_id end) as in_app_june22
    from txn_data
    group by 1
)

, users_saw_home as (
    select 
        CAST(json_extract(context.traits, '$.userId') AS VARCHAR(100)) as user_id
    from default.segment_jupiter_data
    where 
        name = 'Home'
        and CAST(CAST(from_unixtime(cast(dt as bigint)/1000) as timestamp)  as date) 
    between date('2022-05-16') and date('2022-06-15')
    group by 1
),

users_did_txn as (
    select 
        customer_id as user_id
    from 
        transformations_db.jupiter_reconciliation_transaction_master as t1
    where 
        updated_at = (
            select 
                max(updated_at) 
            from transformations_db.jupiter_reconciliation_transaction_master as t2
            where 
                t1.reconciliation_id = t2.reconciliation_id
                and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
                and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
                and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
                and (t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
                    OR (account_id like '%XX%'and t2.reconciliation_status in ('RECONCILIATION_JUPITER_SUCCESS'))
                    )
                    )
        and date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) 
    between date('2022-05-16') and date('2022-06-15')
    group by 1
)

, active_users as (
    select 
    user_id,
    'active' as flg
from (
    select * from users_saw_home
    union
    select * from users_did_txn
    )
group by 1,2
)

select distinct a.user_id, b.flg, b.kyc_mode, c.acc_balance as acc_balance, d.*, e.*,
f.flg as flg
from ob_base a
left join fkyc_data b on a.user_id = b.user_id
left join user_balances c on a.user_id = c.user_id
left join off_app d on a.user_id = d.user_id
left join in_app e on a.user_id = e.user_id
left join active_users f on a.user_id = f.user_id
where a.acc_created_date between date('2021-06-01') and date('2021-06-30')
''',conn)

DatabaseError: Execution failed on sql: 

with ob_base as (
    select *
    from
    (select prospective_user_id as user_id,
           min(date(user_saving_account_created_date)) as acc_created_date
    from transformations_db.jupiter_onboarding_master 
    where user_saving_account_created_date is not null 
    group by 1
    )

)

, fkyc_data as (
    select *, 
    case when time_taken < 30 then 'ETB'
    when time_taken >= 30 then 'VKYC' end as kyc_mode
    from 
    (
    select distinct prospective_user_id as user_id, 'FKYC' as flg,
    user_saving_account_created_date, full_kyc_completion_date,
    date_diff('minute', user_saving_account_created_date, full_kyc_completion_date) as time_taken
    from transformations_db.jupiter_onboarding_master
    where user_saving_account_type = 'jupiter.accounts.full'
    and user_saving_account_created_date is not null and full_kyc_completion_date is not null
    ) 
)

, balance as
(select *
from transformations_db.jupiter_account_balance
w

In [ ]:
%%athena


with recon_transactions as 
(select reconciliation_id,transaction_channel,
transaction_type,transaction_amount,reconciliation_status,cbs_log_payer_particulars,cbs_log_payee_particulars,cbs_transaction_datetime

from transformations_db.jupiter_reconciliation_transaction_master as t1
and updated_at = 
        (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
        where t1.reconciliation_id = t2.reconciliation_id
            and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
            and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
            and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
            and (lower(cbs_log_payee_particulars) not like '%withdraw%from%')
            and (lower(cbs_log_payer_particulars) not like '%transfer%to%')
           and t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
            )     
group by 1,2,3,4,5,6,7,8
)

select count(distinct reconciliation_id)
from recon_transactions
where transaction_type = 'DEPOSIT'
and 

In [5]:
%%athena

drop table active_users_data

In [8]:
%%athena

create table active_users_data as
with users_saw_home as (
    select 
        CAST(json_extract(context.traits, '$.userId') AS VARCHAR(100)) as user_id,
        'home' as flg
    from default.segment_jupiter_data
    where 
        name = 'Home'
        and CAST(CAST(from_unixtime(cast(dt as bigint)/1000) as timestamp)  as date) 
    between date('2022-05-16') and date('2022-06-15')
    group by 1,2
),

, recon_users as 
(
    select customer_id as user_id,
    'txng' as flg
    from
(select *
from transformations_db.jupiter_reconciliation_transaction_master as t1
where updated_at = (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
                   where t1.reconciliation_id = t2.reconciliation_id
and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
and (lower(cbs_log_payee_particulars) not like '%withdraw%from%')
and (lower(cbs_log_payer_particulars) not like '%transfer%to%')
and (t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
        OR (account_id like '%XX%'and t2.reconciliation_status in ('RECONCILIATION_JUPITER_SUCCESS')))
))
)



, hp_active_users as (
    select *
    from users_saw_home 
    where user_id not in (select user_id from recon_users)
)

, txn_users as (
    select *
    from recon_users 
)

select count(distinct user_id)
from
(
    select *
    from hp_active_users
    union
    select *
    from txn_users
)

In [11]:
%%athena

select count(distinct user_id)
from active_users_data
limit 3

,_col0
0,520620


In [13]:
d1 = pd.read_sql('''

with users_saw_home as (
    select 
        CAST(json_extract(context.traits, '$.userId') AS VARCHAR(100)) as user_id
    from default.segment_jupiter_data
    where 
        name = 'Home'
        and CAST(CAST(from_unixtime(cast(dt as bigint)/1000) as timestamp)  as date) 
    between date('2022-05-16') and date('2022-06-15')
    group by 1
)

, recon_users as 
(
    select customer_id as user_id
    from
(select *
from transformations_db.jupiter_reconciliation_transaction_master as t1
where updated_at = (select max(updated_at) from transformations_db.jupiter_reconciliation_transaction_master as t2
                   where t1.reconciliation_id = t2.reconciliation_id
and (payments_log_payer_particulars <> 'Jupiter Rewards' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%transfer%pot%' OR payments_log_payer_particulars is null)
and (lower(payments_log_payer_particulars) not like '%withdraw%pot%' OR payments_log_payer_particulars is null)
and (lower(cbs_log_payee_particulars) not like '%withdraw%from%')
and (lower(cbs_log_payer_particulars) not like '%transfer%to%')
and (t2.reconciliation_status in ('RECONCILIATION_SUCCESSFUL','RECONCILIATION_CBS_INITIATED')
        OR (account_id like '%XX%'and t2.reconciliation_status in ('RECONCILIATION_JUPITER_SUCCESS')))
))
where date(coalesce(cbs_transaction_datetime, payment_transaction_datetime)) between date('2022-05-16')
and date('2022-06-15')
)

, active_users as (
    select 
    *
from (
    select * from users_saw_home
    union
    select * from recon_users
    )
group by 1
)

select *
from active_users''',conn)

In [14]:
len(d1)

680868

In [17]:
d1.head()

,user_id
0,558b577c-8ebb-46ce-9fc8-a3d1c2a29892
1,d5d0510a-4a49-46c8-89f9-64233929411a
2,0867fc0a-c655-4179-b1b1-1d84b2eb8409
3,673c3afc-45d3-4ea0-b3b2-1a8a612c4f3b
4,CWc-RYx-6S9D-633


In [18]:
d1['user_id'].nunique()

680867